<a href="https://colab.research.google.com/github/Lau7an/PoshmarkInventoryAndImageScraper/blob/main/PoshmarkInventoryAndImageScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import json
import time
import random
import base64
from IPython.display import display, HTML
from datetime import date
import os
from google.colab import drive, files
from urllib.request import urlopen
from urllib.parse import urlparse
import os

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Enter the Poshmark Username to be scraped

Poshmark_Username = "chaoscollection"


In [ ]:
def ProPoshScraper(pro):
    print(pro)
    pd.set_option('display.max_columns', 100)

    MIN_SLEEP_TIME = 2
    page = 1

    proUrlBase = "https://poshmark.com/vm-rest/users/"
    proUsername = pro
    proFilter = "/posts/filtered?request=%7B%22filters%22%3A%7B%22"
    proDepartment = "department%22%3A%22All%22%2C%22"
    proInventory = "inventory_status%22%3A%5B%22"
    # all, available, sold_out
    Inventory = "available"
    proURLTail1 = "%22%5D%7D%2C%22query_and_facet_filters%22%3A%7B%22creator_id%22%3A%22"
    proURLTail2 = "%22%7D%2C%22experience%22%3A%22all%22%2C%22"
    proMaxID = "max_id%22%3A"
    MaxIDTail = "%2C%22"
    proCount = "count%22%3A"
    proCount2 = 48
    proURLTail3 = "%7D&summarize=true&app_version=2.55&pm_version=129.0.0"

    start_url = proUrlBase + proUsername + proFilter + proDepartment + proInventory + proURLTail1 + proUsername \
                + proURLTail2 + proCount + str(proCount2) + proURLTail3


    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/"
                             "58.0.3029.110 Safari/537.36 SE 2.X MetaSr 1.0"}

    proxies = {"http": "http://123.207.96.189:80"}

    def get_table_from_url(url):
        response = requests.get(url, proxies=proxies, headers=headers)
        if response.status_code != 200:
            print("Couldn't Connect to Poshmark")
        else:
            text = response.text
            nested_text = json.loads(text)
            # print(nested_text)
            if "statusCode" in nested_text:
                if nested_text["statusCode"] == 404:
                    print("Couldn't find Poshmark Username")
            elif "data" in nested_text:
                df_packet = pd.json_normalize(nested_text['data'])
                # total_items = more_data['total']
                try:
                    if 'more' in nested_text:
                        more_data = nested_text['more']
                        if "next_max_id" in more_data:
                            NewMaxID = str(more_data['next_max_id'])
                            new_url = proUrlBase + proUsername + proFilter + proDepartment + proInventory + proURLTail1 + \
                                      proUsername + proURLTail2 + proMaxID + NewMaxID + MaxIDTail + proCount + \
                                      str(proCount2) + proURLTail3
                        else:
                            new_url = "end"
                    else:
                        new_url = "end"
                except:
                    new_url = "end"

                return df_packet, new_url
    try:
        full_table, next_url = get_table_from_url(start_url)
    except:
        print("Userdata not collected")

    if full_table.size > 0:
        print("Table Size: ",full_table.size)
        while next_url != "end":
            print(str(page) + " " + proUsername + " " + str(full_table.shape[0]) + ' items information collected')
            table_to_add, next_url = get_table_from_url(next_url)
            full_table = pd.concat([full_table, table_to_add])
            sleep_time = MIN_SLEEP_TIME + random.random() * 10
            time.sleep(int(sleep_time))
            page = page + 1

        first_columns = ['id','brand','title','department.display','category','category_v2.display',
                         'category_v2.slug','first_published_at','condition','price','original_price_amount.val',
                         'original_price_amount.currency_code','size_obj.display_with_size_set','size_obj.size_system',
                         'picture_url',
                         'inventory.status','inventory.status_changed_at']
        # Removed column restrictions. To add them back, update this line to: table_min = fulle_table[first_columns]
        table_min = full_table

        output_name = 'Poshmark_' + \
                      proUsername + \
                      "_" + \
                      str(table_min.shape[0]) + \
                      " " + \
                      Inventory + \
                      ' items_' + \
                      str(date.today()) + \
                      '.csv'
        return table_min,output_name
    else:
        print("No userdata collected")


## https://www.codegrepper.com/code-examples/python/download+csv+file+through+jupyter+notebook
def create_download_link( df, file):
    title = "Download CSV file"
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=file)
    display(HTML(html))
    return HTML(html)

def download_csv_to_google_drive(df, file):
  output_directory = "/content/drive/My Drive/Poshmark_Images"
  user_dir = output_directory + '/' + Poshmark_Username
  csv_filename = user_dir + "/" + file
  if not os.path.isfile(user_dir):
    os.makedirs(user_dir, exist_ok=True)
  csv = df.to_csv()
  df.to_csv(csv_filename)

try:
    results, filename = ProPoshScraper(Poshmark_Username)
    download_csv_to_google_drive(df=results,file=filename)
    create_download_link(df=results,file=filename)
except:
    print("No data available for csv output")

shanishrenee
Table Size:  5808
1 shanishrenee 48 items information collected
2 shanishrenee 96 items information collected
3 shanishrenee 144 items information collected


In [ ]:
from google.colab import drive, files
from urllib.request import urlopen
from urllib.parse import urlparse
import os

drive.mount('/content/drive')
output_directory = "/content/drive/My Drive/Poshmark_Images"

available_inventory = results.loc[results['inventory.status'] == 'available']
user_dir = output_directory + '/' + Poshmark_Username

for i, item in available_inventory.iterrows():

  output_dir = output_directory + '/' + Poshmark_Username + "/" + item['id']
  if not os.path.isfile(output_dir):
      os.makedirs(output_dir, exist_ok=True)
  
  pictures = item['pictures']

  for pics in pictures:
    imgs = pics['url_large']
    img = urlparse(imgs)
    name = os.path.basename(img.path)
    data = urlopen(imgs)
    file_dir = output_dir + "/" + name
    print("saving file to: ", file_dir)
    if not os.path.isfile(file_dir):
      with open(file_dir, "wb") as f:
        f.write(data.read())

drive.flush_and_unmount()


Streaming output truncated to the last 5000 lines.
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625a2f2df0f28c3974ebaf82/l_625a2fc8cb692cb4bbd00bfc.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f703fae766f2b30438dc1.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f7150cb692c55daf2a1ba.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f715cf0f28c397409bd1a.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f7164008b99028b7feaca.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f7170d6baab6b2c16b98d.jpg
saving file to:  /content/drive/My Drive/Poshmark_Images/absolute_threds/625f6fcc163df4a09a297ea3/l_625f717aae766f0ec34399bb.jpg
saving file to:  /content/drive/My Drive/Poshm

In [ ]:
from google.colab import drive, files
from urllib.request import urlopen
from urllib.parse import urlparse

import os
import io

drive.mount('/content/drive')
output_directory = "/content/drive/My Drive/Poshmark_Images"
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['4loosethreadspurchase.csv']))
# Dataset is now stored in a Pandas Dataframe
print(df.head())
user_dir = output_directory + '/' + Poshmark_Username

for i, item in df.iterrows():

  output_dir = output_directory + '/' + Poshmark_Username + "/" + item['id']
  if not os.path.isfile(output_dir):
      os.makedirs(output_dir, exist_ok=True)
  
  pictures = item['pictures']

  for pics in pictures:
    imgs = pics['url_large']
    img = urlparse(imgs)
    name = os.path.basename(img.path)
    data = urlopen(imgs)
    file_dir = output_dir + "/" + name
    print("saving file to: ", file_dir)
    if not os.path.isfile(file_dir):
      with open(file_dir, "wb") as f:
        f.write(data.read())
        

drive.flush_and_unmount()

Mounted at /content/drive


Saving 4loosethreadspurchase.csv to 4loosethreadspurchase (2).csv
   Price                        id                brand  \
0   8.58  627574e181a36c79a5fd7841    7 For All Mankind   
1   5.50  61f309acbcbb52e3ef9b0832       89th & Madison   
2  12.98  61b51b44a4de41f03da07583         8igth Dstrkt   
3   6.38  61bfb811920786deecc875f4  Abercrombie & Fitch   
4   6.38  6250e76fae766f07e25bacb5               adidas   

                                               title department.display  \
0  7 For All Mankind Lexie A-Pocket Light Wash Bo...              Women   
1  NWOT 89th + Madison Pale Blue Sparkle Sequin C...              Women   
2  NWT 8igth Dstrkt Size 15/16 red embroidered w/...              Women   
3  Abercrombie & Fitch Navy Blue twisted spaghett...              Women   
4  NWOT ADIDAS Classic 3-Stripe Iconic Satin Back...              Women   

            category category_v2.display category_v2.slug  \
0              Denim               Jeans            Jeans   
1     

KeyError: ignored